In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from  tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd
import shutil
import time
#import torch
#import cv2 as cv2
#from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
#from sklearn.metrics import confusion_matrix, classification_report
from IPython.core.display import display, HTML
from sklearn.metrics import confusion_matrix


In [3]:
c=0
for filename in os.listdir("../archive/images"):
    if filename.endswith(".jpg"):
        c+=1    
print(c)


10015


In [4]:
groundTruth = pd.read_csv('../archive/GroundTruth.csv')

In [44]:
groundTruth

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,label
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
...,...,...,...,...,...,...,...,...,...
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKL
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV


In [6]:
#TRY OVER SAMPLING OTHER LABELS AND UNDER SAMPLING THE NV

#First, add a new column to make over/undersampling easier

def defineLabel(row):
    if row['MEL'] == 1:
        val = "MEL"
    elif row['NV'] ==1:
        val = 'NV'
    elif row['BCC'] ==1:
        val = 'BCC'
    elif row['AKIEC'] ==1:
        val = 'AKIEC'
    elif row['BKL'] ==1:
        val = 'BKL'
    elif row['DF'] ==1:
        val = 'DF'
    else:
        val = 'VASC'
    return val

groundTruth['label'] = groundTruth.apply(defineLabel, axis=1)


In [7]:
groundTruth

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,label
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
...,...,...,...,...,...,...,...,...,...
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0,BKL
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NV


In [8]:
#Check to see if the categories are mutually exclusive
sumRows = groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=1) 
#Add up each row. If mutually exclusive sum of each row should be 1

print(sumRows.sum(axis=0))
print(len(sumRows))

10015.0
10015


In [26]:
sumColumns = pd.DataFrame(groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
sumColumns = sumColumns.rename(columns={0: "Total"})
sumColumns['Percent Total'] = sumColumns['Total'] / len(groundTruth)

In [27]:
sumColumns
#Very skewed data. Will need to do stratified

,Total,Percent Total
MEL,1113.0,0.111133
NV,6705.0,0.669496
BCC,514.0,0.051323
AKIEC,327.0,0.032651
BKL,1099.0,0.109735
DF,115.0,0.011483
VASC,142.0,0.014179


In [11]:
from imblearn import under_sampling, over_sampling
from imblearn.under_sampling import RandomUnderSampler

In [17]:

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(groundTruth, groundTruth["label"])
# print(sorted(Counter(y_resampled).items()))

In [22]:
X_resampled = X_resampled.drop(columns="label")

In [15]:
sumColumns = pd.DataFrame(X_resampled[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
sumColumns = sumColumns.rename(columns={0: "Total"})
sumColumns['Percent Total'] = sumColumns['Total'] / len(groundTruth)
sumColumns

,Total,Percent Total
MEL,115.0,0.011483
NV,115.0,0.011483
BCC,115.0,0.011483
AKIEC,115.0,0.011483
BKL,115.0,0.011483
DF,115.0,0.011483
VASC,115.0,0.011483


In [31]:
#trainingDf, testingDf= train_test_split(groundTruth, test_size=0.30, random_state=42, stratify = groundTruth[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]])
#Stratify on each outcome

trainingDf, testingDf  = train_test_split(X_resampled, test_size=0.3, random_state=42, stratify = X_resampled[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]])
#Stratify on each outcome

#Training DF will be 40% of the data set
#Testing data set will be split again to make a validation and testing data set, (30% each)





In [32]:
trainingDf

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
483,ISIC_0028412,1.0,0.0,0.0,0.0,0.0,0.0,0.0
542,ISIC_0027604,1.0,0.0,0.0,0.0,0.0,0.0,0.0
633,ISIC_0028160,0.0,1.0,0.0,0.0,0.0,0.0,0.0
93,ISIC_0027084,0.0,0.0,0.0,1.0,0.0,0.0,0.0
587,ISIC_0028683,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
383,ISIC_0027613,0.0,0.0,0.0,0.0,0.0,1.0,0.0
771,ISIC_0032614,0.0,0.0,0.0,0.0,0.0,0.0,1.0
452,ISIC_0033790,0.0,0.0,0.0,0.0,0.0,1.0,0.0
544,ISIC_0033129,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Double check to see if stratification worked on the train set
trainSumColumns = pd.DataFrame(trainingDf[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
trainSumColumns = trainSumColumns.rename(columns={0: "Total"})
trainSumColumns['Percent Total'] = trainSumColumns['Total'] / len(trainingDf)
trainSumColumns

,Total,Percent Total
MEL,80.0,0.142096
NV,81.0,0.143872
BCC,80.0,0.142096
AKIEC,80.0,0.142096
BKL,80.0,0.142096
DF,81.0,0.143872
VASC,81.0,0.143872


In [84]:
# Double check to see if stratification worked on the test set
testSumColumns = pd.DataFrame(testingDf[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
testSumColumns = testSumColumns.rename(columns={0: "Total"})
testSumColumns['Percent Total'] = testSumColumns['Total'] / len(testingDf)
testSumColumns

,Total,Percent Total
MEL,1098.0,0.111089
NV,6617.0,0.669466
BCC,513.0,0.051902
AKIEC,323.0,0.032679
BKL,1081.0,0.109369
DF,111.0,0.011230
VASC,141.0,0.014265


In [102]:
# trainingDf = trainingDf[["image", "NV"]]
# testingDf = testingDf[["image", "NV"]]

In [66]:
#Just a simple test data set for setting up the models
train = trainingDf.iloc[1:15,:]

test = testingDf.iloc[1:10,:]

In [34]:
# Make a list to store all of the processed images
x_data_train = []
# y_d_test = []


#Process all the images in the training data set.
for i in trainingDf.iloc[:,0]: #FIXME CHANGE BACK TO TRAINING DF
    path="../archive/images/" + i + ".jpg"
    
    image=tf.keras.preprocessing.image.load_img(
        path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest")
    input_arr = tf.keras.preprocessing.image.img_to_array(image)

    x_data_train.append(input_arr)
    
#     print(trainingDf.loc[trainingDf['image'] == i]["NV"])
#     y_d_test.append(trainingDf.loc[trainingDf['image'] == i]["NV"])




In [ ]:
x_data_test = []

for i in test.iloc[:,0]: #FIXME CHANGE BACK TO TESTING DF
    path="../archive/images/" + i + ".jpg"
    
    image=tf.keras.preprocessing.image.load_img(
        path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest")
    input_arr = tf.keras.preprocessing.image.img_to_array(image)

    x_data_test.append(input_arr)

In [35]:
x_data_train[0].shape


(450, 600, 3)

In [36]:
#Start Building model

shape = x_data_train[0].shape

In [37]:
# Simple dummy model
def buildModel():
    model = keras.models.Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu", input_shape=shape))
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(rate=0.5))
    model.add(Dense(7, activation="softmax"))
    return model
    


In [59]:
m = buildModel()
m.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_34 (Conv2D)          (None, 446, 596, 32)      2432      
                                                                 
 conv2d_35 (Conv2D)          (None, 442, 592, 32)      25632     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 221, 296, 32)     0         
 g2D)                                                            
                                                                 
 dropout_34 (Dropout)        (None, 221, 296, 32)      0         
                                                                 
 flatten_17 (Flatten)        (None, 2093312)           0         
                                                                 
 dense_34 (Dense)            (None, 256)               535888128 
                                                     

In [60]:
# sgd = tf.keras.optimizers.SGD(learning_rate=0.1)
m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [56]:


#Get y-Data ready for model
y_train_final = np.asarray(trainingDf.loc[:, trainingDf.columns != 'image'])
# y_train_final = np.asarray(trainingDf.loc[:, test.columns != 'image'])
print(y_train_final.shape)


#Get x_Data ready for model
x_train_final = np.asarray(x_data_train).astype('float32') / 255.
print(x_train_final.shape) #x_train





(563, 7)
(563, 450, 600, 3)


In [57]:
# from sklearn.model_selection import RandomizedSearchCV

# wrap_model = keras.wrappers.scikit_learn.KerasClassifier(buildModel)

# params = {
#     "epochs": [5, 10],
#     "batch_size": [5, 10,25, 50,100]   
# }

# rnd_search_cv = RandomizedSearchCV(wrap_model, params, n_iter=5, cv=3)
# rnd_search_cv.fit(X = x_data_final, y = y_train_final)

In [61]:
#epochs = 15
history = m.fit(x_train_final, y_train_final, epochs=15, batch_size=64)

#look for keras tunner

Epoch 1/15
9/9 [==============================] - 565s 58s/step - loss: 44.7246 - accuracy: 0.1563
Epoch 2/15
9/9 [==============================] - 519s 57s/step - loss: 1.9566 - accuracy: 0.2433
Epoch 3/15
9/9 [==============================] - 555s 61s/step - loss: 2.1039 - accuracy: 0.3144
Epoch 4/15
9/9 [==============================] - 576s 64s/step - loss: 1.9820 - accuracy: 0.3712
Epoch 5/15
9/9 [==============================] - 554s 62s/step - loss: 1.7500 - accuracy: 0.4281
Epoch 6/15
9/9 [==============================] - 586s 65s/step - loss: 1.6595 - accuracy: 0.4405
Epoch 7/15
9/9 [==============================] - 556s 61s/step - loss: 1.5225 - accuracy: 0.4725
Epoch 8/15
9/9 [==============================] - 563s 62s/step - loss: 1.3801 - accuracy: 0.5453
Epoch 9/15
9/9 [==============================] - 562s 63s/step - loss: 1.2128 - accuracy: 0.6039
Epoch 10/15
9/9 [==============================] - 568s 62s/step - loss: 1.1680 - accuracy: 0.6714
Epoch 11/15
9/9 [=

In [90]:
m.optimizer.get_config()

{'name': 'Adam',
 'learning_rate': 0.001,
 'decay': 0.0,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [91]:
history.params

{'verbose': 1, 'epochs': 15, 'steps': 9}

In [2]:
history.prams

NameError: name 'history' is not defined

In [47]:
x_train_final

array([[[[0.34901962, 0.2       , 0.24313726],
         [0.36078432, 0.21176471, 0.2627451 ],
         [0.36862746, 0.21960784, 0.27058825],
         ...,
         [0.14901961, 0.08235294, 0.11372549],
         [0.13725491, 0.07843138, 0.10588235],
         [0.13725491, 0.07450981, 0.11372549]],

        [[0.37254903, 0.22352941, 0.27450982],
         [0.3882353 , 0.22745098, 0.28235295],
         [0.36862746, 0.22745098, 0.27450982],
         ...,
         [0.14117648, 0.07843138, 0.12156863],
         [0.14509805, 0.08235294, 0.12156863],
         [0.13725491, 0.07450981, 0.11372549]],

        [[0.3882353 , 0.23921569, 0.2901961 ],
         [0.39607844, 0.23529412, 0.2901961 ],
         [0.39215687, 0.24313726, 0.29411766],
         ...,
         [0.14901961, 0.08627451, 0.1254902 ],
         [0.14509805, 0.08627451, 0.11372549],
         [0.13725491, 0.07450981, 0.11372549]],

        ...,

        [[0.38431373, 0.29411766, 0.3254902 ],
         [0.39607844, 0.3137255 , 0.34117648]

In [62]:
test1, test2  = train_test_split( trainingDf, test_size=0.01, random_state=42, stratify = trainingDf[["NV"]])
# trainingDf

In [63]:
test1

tc = pd.DataFrame(test1[["MEL","NV","BCC","AKIEC","BKL","DF","VASC"]].sum(axis=0))
tc = tc.rename(columns={0: "Total"})
tc['Percent Total'] = tc['Total'] / len(testingDf)
tc

,Total,Percent Total
MEL,80.0,0.330579
NV,80.0,0.330579
BCC,79.0,0.326446
AKIEC,80.0,0.330579
BKL,79.0,0.326446
DF,80.0,0.330579
VASC,79.0,0.326446


In [77]:
# Make a list to store all of the processed images
XTEST = []
# y_d_test = []


#Process all the images in the training data set.
for i in testingDf.iloc[:,0]: #FIXME CHANGE BACK TO TRAINING DF
    path="../archive/images/" + i + ".jpg"
    
    image=tf.keras.preprocessing.image.load_img(
        path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest")
    input_arr = tf.keras.preprocessing.image.img_to_array(image)

    XTEST.append(input_arr)
    
#     print(trainingDf.loc[trainingDf['image'] == i]["NV"])
#     y_d_test.append(trainingDf.loc[trainingDf['image'] == i]["NV"])



In [78]:
#Get y-Data ready for model
YTEST = np.asarray(testingDf.loc[:, testingDf.columns != 'image'])
# y_train_final = np.asarray(trainingDf.loc[:, test.columns != 'image'])
print(YTEST.shape)


#Get x_Data ready for model
XTEST = np.asarray(XTEST).astype('float32') / 255.
print(XTEST.shape) #x_train



(242, 7)
(242, 450, 600, 3)


In [79]:
predictions = m.predict(XTEST)



In [80]:
y_pred = m.predict(XTEST)
y_pred = np.argmax(y_pred, axis=1)





In [81]:
y_pred

array([1, 6, 6, 1, 0, 4, 6, 1, 1, 1, 6, 5, 6, 1, 1, 1, 4, 4, 1, 0, 3, 5,
       2, 3, 2, 1, 4, 6, 0, 4, 1, 3, 1, 3, 4, 1, 5, 6, 2, 5, 0, 4, 1, 1,
       6, 4, 5, 0, 1, 3, 2, 4, 4, 4, 3, 4, 1, 5, 1, 4, 1, 1, 4, 1, 1, 4,
       2, 0, 0, 1, 3, 5, 0, 1, 0, 5, 1, 0, 0, 4, 1, 5, 4, 1, 1, 3, 1, 5,
       5, 4, 0, 1, 2, 1, 1, 5, 6, 1, 4, 1, 4, 4, 1, 5, 0, 3, 2, 1, 0, 1,
       3, 1, 0, 5, 5, 3, 0, 1, 6, 3, 6, 0, 1, 6, 0, 0, 1, 3, 5, 4, 5, 4,
       4, 1, 6, 1, 4, 4, 1, 1, 2, 1, 3, 1, 4, 2, 1, 4, 2, 1, 1, 0, 5, 6,
       1, 1, 1, 0, 2, 4, 6, 6, 6, 1, 0, 4, 1, 1, 0, 1, 2, 1, 3, 1, 4, 6,
       4, 0, 2, 4, 2, 4, 4, 4, 1, 0, 4, 0, 6, 6, 1, 1, 0, 1, 2, 2, 5, 0,
       6, 2, 5, 2, 2, 2, 6, 4, 0, 1, 1, 4, 1, 4, 1, 5, 1, 5, 1, 6, 3, 1,
       5, 1, 4, 3, 1, 4, 1, 2, 0, 5, 3, 1, 3, 0, 3, 1, 6, 0, 5, 1, 3, 0])

In [82]:
truth = []

for i in YTEST:
    if np.array_equal(i, np.asarray([1,0,0,0,0,0,0])):
        truth.append(0)
    if np.array_equal(i, np.asarray([0,1,0,0,0,0,0])):
        truth.append(1)
    if np.array_equal(i, np.asarray([0,0,1,0,0,0,0])):
        truth.append(2)
    if np.array_equal(i, np.asarray([0,0,0,1,0,0,0])):
        truth.append(3)
    if np.array_equal(i, np.asarray([0,0,0,0,1,0,0])):
        truth.append(4)
    if np.array_equal(i, np.asarray([0,0,0,0,0,1,0])):
        truth.append(5)
    if np.array_equal(i, np.asarray([0,0,0,0,0,0,1])):
        truth.append(6)
        
truth = np.asarray(truth)
  

In [83]:
conf_mat = confusion_matrix(y_pred, truth)


In [84]:
conf_mat

array([[ 7,  7,  3,  6,  7,  1,  2],
       [21, 25,  6,  7, 10,  3,  4],
       [ 0,  0,  5,  4,  2, 10,  0],
       [ 1,  1,  3,  5,  3,  7,  1],
       [ 5,  1, 11,  7,  8,  6,  4],
       [ 1,  0,  7,  6,  4,  7,  0],
       [ 0,  0,  0,  0,  1,  0, 23]])

### Autoencoder

In [ ]:
#Process test data
x_data_final_test = np.asarray(x_data_test)
x_test = x_data_final_test.astype('float32') / 255.
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

NameError: name 'x_test' is not defined

In [ ]:
x_test.shape

In [ ]:
encoding_dim = 64  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(810000,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(810000, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [ ]:
# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

In [ ]:
#This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))

# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]

# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))


In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                verbose = False,
                validation_data=(x_test, x_test))

In [ ]:
# Encode and decode some digits
# Note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


In [ ]:
# Use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 9  # How many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(450, 600,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(450, 600,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### Autoencoder for denoising.

In [ ]:
x_train_new = x_data_final_train.astype('float32') / 255.
x_test_new = x_data_final_test.astype('float32') / 255.
x_train_new = np.reshape(x_train_new, (len(x_train), 450, 600, 3))
x_test_new = np.reshape(x_test_new, (len(x_test), 450, 600, 3))

noise_factor = 0.5
x_train_noisy = x_train_new + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train_new.shape) 
x_test_noisy = x_test_new + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test_new.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
x_train_noisy.shape

In [ ]:
x_train_new.shape

In [ ]:
n = 8
plt.figure(figsize=(20, 2))
for i in range(1, n + 1):
    ax = plt.subplot(1, n, i)
    plt.imshow(x_test_noisy[i].reshape(450, 600, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
from tensorflow.keras import layers

input_img = keras.Input(shape=(450, 600, 3))

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(x_train_noisy, x_train_new,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test_new))

In [ ]:
decoded_imgs = autoencoder.predict(x_test_noisy)

n = 8
plt.figure(figsize=(20, 4))
for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test_noisy[i].reshape(450, 600, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(450, 600, 3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()